# Parallel code for non-isomorphic version A364656: Number of inequivalent strict interval closure operators on a set of n elements. 

All the values do not count closure system {{},[n]} since it forms a reducible context; so, we need to add 1 to the obtained results.

In [9]:
%load_ext Cython


The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


## 1. Post-filtering approach (up to n=5)

In [10]:
%%cython --annotate
from math import log 

cpdef ProcessRT(t,int i,int l,int nn1):
    """This function recursively extends an input tuple of integers, which represents 
    a set of sets (more precisily, a formal context), by adding to it an integer i (a new set).
    The length of a terminal tuple is constrained by l, while nn1 should be passed as 2**n-1. 
    The function checks whether the new tuple forms a unique closure system and returns the dictionary
    of the first elements of processed tuples as keys and values 1 for clousre system with
    interval closure propery and 0 otherwise."""
    
    count={}
    t=t[:]
    cdef int P
    cdef int lent=len(t)

    
    #checking whether the context is reducible
    for j in range(lent-1,-1,-1):
        P=i
        #print("t[j],i",t[j],i,t[j]&i,)
        tji=t[j]&i
        if tji in t[0:j]:
            return count
            
        if tji==t[j]:
            #print("t[j],i",t[j],i,t[j]&i,)
            for h in range(j+1,lent):
                if t[j]&t[h]==t[j]:
                    P=P&t[h]
                    if P==t[j]:                        
                        return count
    t.append(i)

 
    if Check(t,nn1):
        count[(lent+1,tuple(t))]=1
    #else:
    #    count[(lent+1,t[0])]=0

    #recursive call
    cdef unsigned long long r
    #print(l)
    if l!=0:
        r=0
        for k in range(i+1,nn1):
            #print("Process(t,k,l-1,tuples,nn1)")
            #print("t",t,"k",k)
            ucount=ProcessRT(t,k,l-1,nn1)
            for key in ucount:
                if key in count:
                    count[key]+=ucount[key]
                else:
                    count[key]=ucount[key]
            #if r<0: print("r",r)
        return count
    else: 
        #print(tuples)
        #tuples.append(tuple(t))
       
        
        print("Branch process",t[0])
        return count

    
def isClosed(t,g):
    """Checks whether g is closed in t."""
    cont=[]
    for h in t:
        if g&h==g:
            cont.append(h)
    if len(cont)>0:
        res=cont[0]
        for h in cont:
            res=h&res
        return res==g
    else:
        return g==0
    
def Closure(t,g):
    """Returns closure of g in t."""
    res=~0
    for h in t:
        if g&h==g:
            res=h&res
    return res
    
    
def Check(t,nn1):
    """Checks whether {x,y}'' is subset of S is fulfilled for t."""
    t=t+[nn1]
    #print(t)
    n=int(log(nn1+1,2))
    members=[]
    members.extend([2**i for i in range(n)])
    powerset=range(nn1)
    lm=len(members)
    pairs=[members[i]|members[j] for i in range(lm) for j in range(i+1,lm)]
    closures={}
    for p in pairs:
        closures[p]=Closure(t,p)
    for s in powerset:
        mode=True
#         for i in range(lm):
#             for j in range(i+1,lm):
#                 p=members[i]|members[j]
        for p in pairs:   
            #cl=Closure(t,p)
            if p&s==p:
                if closures[p]&s!=closures[p]:
                        #print("False")
                        #print(p,cl,s)
                    mode=False
                    break
            #if mode==False: break           
                        
        if mode:
            if s not in members and Closure(t,s)!=s:
                #print("False")
                #print(t,s)
                #print(Closure(t,s))
                return False
                        
            
    return True


In [11]:
def merge(r):
    """This is a value collector for the resulting list of dictionaries
    """
    merged={}
    for dic in r:
        for key in dic:
            if key in merged:
                merged[key]+=dic[key]
            else:
                merged[key]=dic[key]
    return merged



In [12]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count


n=2

#computing A334254 for n=6 by levels
if __name__ == "__main__":
    pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
   
     
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res2 = pool.starmap_async(ProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res2.get())# print the list of resulting dictionaries
    print(sum(merge(res2.get()).values()))#print the number of closure systems w.r.t. T1
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 2, 3), ([0], 2, 2, 3), ([1], 2, 2, 3)]
[{(2, (0, 1)): 1}, {(2, (0, 2)): 1}, {(2, (1, 2)): 1}]
3
CPU times: user 0 ns, sys: 45.2 ms, total: 45.2 ms
Wall time: 55.8 ms


In [13]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count


n=3

#computing A334254 for n=6 by levels
if __name__ == "__main__":
    pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
   
     
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res3 = pool.starmap_async(ProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res3.get())# print the list of resulting dictionaries
    print(sum(merge(res3.get()).values()))#print the number of closure systems w.r.t. T1
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 6, 7), ([0], 2, 6, 7), ([0], 3, 6, 7), ([0], 4, 6, 7), ([0], 5, 6, 7), ([0], 6, 6, 7), ([1], 2, 6, 7), ([1], 3, 6, 7), ([1], 4, 6, 7), ([1], 5, 6, 7), ([1], 6, 6, 7), ([2], 3, 6, 7), ([2], 4, 6, 7), ([2], 5, 6, 7), ([2], 6, 6, 7), ([3], 4, 6, 7), ([3], 5, 6, 7), ([3], 6, 6, 7), ([4], 5, 6, 7), ([4], 6, 6, 7), ([5], 6, 6, 7)]
[{(2, (0, 1)): 1, (3, (0, 1, 3)): 1, (3, (0, 1, 5)): 1}, {(2, (0, 2)): 1, (3, (0, 2, 3)): 1, (3, (0, 2, 6)): 1}, {(2, (0, 3)): 1, (3, (0, 3, 5)): 1, (3, (0, 3, 6)): 1}, {(2, (0, 4)): 1, (3, (0, 4, 5)): 1, (3, (0, 4, 6)): 1}, {(2, (0, 5)): 1, (3, (0, 5, 6)): 1}, {(2, (0, 6)): 1}, {(2, (1, 2)): 1, (3, (1, 2, 3)): 1, (4, (1, 2, 3, 4)): 1, (3, (1, 2, 4)): 1, (4, (1, 2, 4, 5)): 1, (4, (1, 2, 4, 6)): 1, (3, (1, 2, 5)): 1, (4, (1, 2, 5, 6)): 1, (3, (1, 2, 6)): 1}, {(3, (1, 3, 4)): 1, (4, (1, 3, 4, 6)): 1, (3, (1, 3, 6)): 1}, {(2, (1, 4)): 1, (3, (1, 4, 5)): 1, (3, (1, 4, 6)): 1}, {(3, (1, 5, 6)): 1}, {(2, (1, 6)): 1}, {(3, (2, 3, 4)): 1, (4, (2, 3, 4, 5)): 1, (3

In [14]:
res3.get()

[{(2, (0, 1)): 1, (3, (0, 1, 3)): 1, (3, (0, 1, 5)): 1},
 {(2, (0, 2)): 1, (3, (0, 2, 3)): 1, (3, (0, 2, 6)): 1},
 {(2, (0, 3)): 1, (3, (0, 3, 5)): 1, (3, (0, 3, 6)): 1},
 {(2, (0, 4)): 1, (3, (0, 4, 5)): 1, (3, (0, 4, 6)): 1},
 {(2, (0, 5)): 1, (3, (0, 5, 6)): 1},
 {(2, (0, 6)): 1},
 {(2, (1, 2)): 1,
  (3, (1, 2, 3)): 1,
  (4, (1, 2, 3, 4)): 1,
  (3, (1, 2, 4)): 1,
  (4, (1, 2, 4, 5)): 1,
  (4, (1, 2, 4, 6)): 1,
  (3, (1, 2, 5)): 1,
  (4, (1, 2, 5, 6)): 1,
  (3, (1, 2, 6)): 1},
 {(3, (1, 3, 4)): 1, (4, (1, 3, 4, 6)): 1, (3, (1, 3, 6)): 1},
 {(2, (1, 4)): 1, (3, (1, 4, 5)): 1, (3, (1, 4, 6)): 1},
 {(3, (1, 5, 6)): 1},
 {(2, (1, 6)): 1},
 {(3, (2, 3, 4)): 1, (4, (2, 3, 4, 5)): 1, (3, (2, 3, 5)): 1},
 {(2, (2, 4)): 1, (3, (2, 4, 5)): 1, (3, (2, 4, 6)): 1},
 {(2, (2, 5)): 1, (3, (2, 5, 6)): 1},
 {},
 {(2, (3, 4)): 1, (3, (3, 4, 5)): 1, (3, (3, 4, 6)): 1},
 {(3, (3, 5, 6)): 1},
 {},
 {},
 {},
 {}]

In [15]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count


n=4

#computing A334254 for n=6 by levels
if __name__ == "__main__":
    pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
   
     
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res4 = pool.starmap_async(ProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res4.get())# print the list of resulting dictionaries
    print(sum(merge(res4.get()).values()))#print the number of closure systems w.r.t. T1
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 14, 15), ([0], 2, 14, 15), ([0], 3, 14, 15), ([0], 4, 14, 15), ([0], 5, 14, 15), ([0], 6, 14, 15), ([0], 7, 14, 15), ([0], 8, 14, 15), ([0], 9, 14, 15), ([0], 10, 14, 15), ([0], 11, 14, 15), ([0], 12, 14, 15), ([0], 13, 14, 15), ([0], 14, 14, 15), ([1], 2, 14, 15), ([1], 3, 14, 15), ([1], 4, 14, 15), ([1], 5, 14, 15), ([1], 6, 14, 15), ([1], 7, 14, 15), ([1], 8, 14, 15), ([1], 9, 14, 15), ([1], 10, 14, 15), ([1], 11, 14, 15), ([1], 12, 14, 15), ([1], 13, 14, 15), ([1], 14, 14, 15), ([2], 3, 14, 15), ([2], 4, 14, 15), ([2], 5, 14, 15), ([2], 6, 14, 15), ([2], 7, 14, 15), ([2], 8, 14, 15), ([2], 9, 14, 15), ([2], 10, 14, 15), ([2], 11, 14, 15), ([2], 12, 14, 15), ([2], 13, 14, 15), ([2], 14, 14, 15), ([3], 4, 14, 15), ([3], 5, 14, 15), ([3], 6, 14, 15), ([3], 7, 14, 15), ([3], 8, 14, 15), ([3], 9, 14, 15), ([3], 10, 14, 15), ([3], 11, 14, 15), ([3], 12, 14, 15), ([3], 13, 14, 15), ([3], 14, 14, 15), ([4], 5, 14, 15), ([4], 6, 14, 15), ([4], 7, 14, 15), ([4], 8, 14, 15), ([4], 9

In [16]:
res4.get()

[{(2, (0, 1)): 1,
  (3, (0, 1, 3)): 1,
  (4, (0, 1, 3, 7)): 1,
  (4, (0, 1, 3, 11)): 1,
  (3, (0, 1, 5)): 1,
  (4, (0, 1, 5, 7)): 1,
  (4, (0, 1, 5, 13)): 1,
  (3, (0, 1, 7)): 1,
  (4, (0, 1, 7, 11)): 1,
  (4, (0, 1, 7, 13)): 1,
  (3, (0, 1, 9)): 1,
  (4, (0, 1, 9, 11)): 1,
  (4, (0, 1, 9, 13)): 1,
  (3, (0, 1, 11)): 1,
  (4, (0, 1, 11, 13)): 1,
  (3, (0, 1, 13)): 1},
 {(2, (0, 2)): 1,
  (3, (0, 2, 3)): 1,
  (4, (0, 2, 3, 7)): 1,
  (4, (0, 2, 3, 11)): 1,
  (3, (0, 2, 6)): 1,
  (4, (0, 2, 6, 7)): 1,
  (4, (0, 2, 6, 14)): 1,
  (3, (0, 2, 7)): 1,
  (4, (0, 2, 7, 11)): 1,
  (4, (0, 2, 7, 14)): 1,
  (3, (0, 2, 10)): 1,
  (4, (0, 2, 10, 11)): 1,
  (4, (0, 2, 10, 14)): 1,
  (3, (0, 2, 11)): 1,
  (4, (0, 2, 11, 14)): 1,
  (3, (0, 2, 14)): 1},
 {(2, (0, 3)): 1,
  (3, (0, 3, 5)): 1,
  (4, (0, 3, 5, 7)): 1,
  (5, (0, 3, 5, 7, 9)): 1,
  (4, (0, 3, 5, 9)): 1,
  (5, (0, 3, 5, 9, 11)): 1,
  (5, (0, 3, 5, 9, 13)): 1,
  (4, (0, 3, 5, 11)): 1,
  (5, (0, 3, 5, 11, 13)): 1,
  (4, (0, 3, 5, 13)): 1,
  (3, 

In [17]:
len(merge(res4.get()).values())

2061

In [18]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count


n=5

#computing A334254 for n=6 by levels
if __name__ == "__main__":
    pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
   
     
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res5 = pool.starmap_async(ProcessRT, zip(lt,lk,ln_2,lnn1))
    #print(res5.get())# print the list of resulting dictionaries
    print(sum(merge(res5.get()).values()))#print the number of closure systems w.r.t. T1
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 30, 31), ([0], 2, 30, 31), ([0], 3, 30, 31), ([0], 4, 30, 31), ([0], 5, 30, 31), ([0], 6, 30, 31), ([0], 7, 30, 31), ([0], 8, 30, 31), ([0], 9, 30, 31), ([0], 10, 30, 31), ([0], 11, 30, 31), ([0], 12, 30, 31), ([0], 13, 30, 31), ([0], 14, 30, 31), ([0], 15, 30, 31), ([0], 16, 30, 31), ([0], 17, 30, 31), ([0], 18, 30, 31), ([0], 19, 30, 31), ([0], 20, 30, 31), ([0], 21, 30, 31), ([0], 22, 30, 31), ([0], 23, 30, 31), ([0], 24, 30, 31), ([0], 25, 30, 31), ([0], 26, 30, 31), ([0], 27, 30, 31), ([0], 28, 30, 31), ([0], 29, 30, 31), ([0], 30, 30, 31), ([1], 2, 30, 31), ([1], 3, 30, 31), ([1], 4, 30, 31), ([1], 5, 30, 31), ([1], 6, 30, 31), ([1], 7, 30, 31), ([1], 8, 30, 31), ([1], 9, 30, 31), ([1], 10, 30, 31), ([1], 11, 30, 31), ([1], 12, 30, 31), ([1], 13, 30, 31), ([1], 14, 30, 31), ([1], 15, 30, 31), ([1], 16, 30, 31), ([1], 17, 30, 31), ([1], 18, 30, 31), ([1], 19, 30, 31), ([1], 20, 30, 31), ([1], 21, 30, 31), ([1], 22, 30, 31), ([1], 23, 30, 31), ([1], 24, 30, 31), ([1], 25,

589601
CPU times: user 387 ms, sys: 16.7 ms, total: 403 ms
Wall time: 3.19 s


In [43]:
def PermBit(bit,perm):
    out=0
    for i in range(len(perm)):
        if (bit>>i) & 1==1:
            out=out | (1<<perm[i])
    return out

def PermTuple(t,perm):
    ret=[]
    for bit in t:
        ret.append(PermBit(bit,perm))
    return tuple(ret)


def isLectLess(a,b):
    #is A lectically smaller than B?
    for i in range(len(a)):
        if a[i]==b[i]:
            pass
        elif a[i]<b[i]:
            return True
        else:
            return False
    return True
        
def num2list(n):
    ret=[]
    cnt=0
    t=n
    while t!=0:
        print((n&(2**cnt))>>cnt)
        if (n&(2**cnt))>>cnt==1:
            ret.append(cnt)
        t=t>>1
        cnt+=1
    return ret
def list2fam(L):
    return [num2list(n) for n in L]

In [46]:
list2fam([0,1])

1


[[], [0]]

In [37]:
7>>2

1

In [20]:
from itertools import permutations


n=2

Perms=list(permutations(range(n)))

d2=merge(res2.get())


T2=[]
for key in d2.keys():
    T2.append(key[1])


ineqT2=[]
for t in T2:
    mode=True
    for p in Perms:
        pt=sorted(PermTuple(t,p))
        if isLectLess(t,pt):
            pass
        else:
            mode=False
            break
            
            
    if mode: 
        ineqT2.append(t)

In [21]:
ineqT2

[(0, 1), (1, 2)]

In [22]:
from itertools import permutations


n=3

Perms=list(permutations(range(n)))

d3=merge(res3.get())


T3=[]
for key in d3.keys():
    T3.append(key[1])


ineqT3=[]
for t in T3:
    mode=True
    for p in Perms:
        pt=sorted(PermTuple(t,p))
        if isLectLess(t,pt):
            pass
        else:
            mode=False
            break
            
            
    if mode: 
        ineqT3.append(t)

In [23]:
ineqT3

[(0, 1),
 (0, 1, 3),
 (0, 3),
 (0, 3, 5),
 (1, 2),
 (1, 2, 3),
 (1, 2, 3, 4),
 (1, 2, 4),
 (1, 2, 5),
 (1, 2, 5, 6),
 (1, 3, 6),
 (1, 6),
 (3, 5, 6)]

In [48]:
[list2fam(t) for t in ineqT3]

1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
0
1
1
1
1
0
1
1
1
0
0
1
1
0
1
0
0
1
1
0
1
1
0
1
1
0
1
1
0
1
0
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
0
1
0
1
1


[[[], [0]],
 [[], [0], [0, 1]],
 [[], [0, 1]],
 [[], [0, 1], [0, 2]],
 [[0], [1]],
 [[0], [1], [0, 1]],
 [[0], [1], [0, 1], [2]],
 [[0], [1], [2]],
 [[0], [1], [0, 2]],
 [[0], [1], [0, 2], [1, 2]],
 [[0], [0, 1], [1, 2]],
 [[0], [1, 2]],
 [[0, 1], [0, 2], [1, 2]]]

In [24]:
from itertools import permutations


n=4

Perms=list(permutations(range(n)))

d4=merge(res4.get())


T4=[]
for key in d4.keys():
    T4.append(key[1])


ineqT4=[]
for t in T4:
    mode=True
    for p in Perms:
        pt=sorted(PermTuple(t,p))
        if isLectLess(t,pt):
            pass
        else:
            mode=False
            break
            
            
    if mode: 
        ineqT4.append(t)
              

In [25]:
ineqT4

[(0, 1),
 (0, 1, 3),
 (0, 1, 3, 7),
 (0, 1, 7),
 (0, 1, 7, 11),
 (0, 3),
 (0, 3, 5),
 (0, 3, 5, 7),
 (0, 3, 5, 7, 9),
 (0, 3, 5, 9),
 (0, 3, 5, 11),
 (0, 3, 5, 11, 13),
 (0, 3, 7),
 (0, 3, 7, 13),
 (0, 3, 13),
 (0, 7),
 (0, 7, 11),
 (0, 7, 11, 13),
 (1, 2),
 (1, 2, 3),
 (1, 2, 3, 4),
 (1, 2, 3, 4, 7),
 (1, 2, 3, 4, 7, 8),
 (1, 2, 3, 4, 8),
 (1, 2, 3, 4, 8, 12),
 (1, 2, 3, 4, 11),
 (1, 2, 3, 4, 11, 12),
 (1, 2, 3, 4, 12),
 (1, 2, 3, 7),
 (1, 2, 3, 7, 12),
 (1, 2, 3, 12),
 (1, 2, 4),
 (1, 2, 4, 7),
 (1, 2, 4, 7, 8),
 (1, 2, 4, 7, 8, 11),
 (1, 2, 4, 7, 11),
 (1, 2, 4, 8),
 (1, 2, 4, 9),
 (1, 2, 4, 9, 10),
 (1, 2, 4, 9, 10, 11),
 (1, 2, 4, 9, 10, 11, 12),
 (1, 2, 4, 9, 10, 12),
 (1, 2, 4, 9, 10, 13),
 (1, 2, 4, 9, 10, 13, 14),
 (1, 2, 4, 9, 11),
 (1, 2, 4, 9, 11, 14),
 (1, 2, 4, 9, 14),
 (1, 2, 4, 11),
 (1, 2, 4, 11, 13),
 (1, 2, 4, 11, 13, 14),
 (1, 2, 5),
 (1, 2, 5, 6),
 (1, 2, 5, 6, 7),
 (1, 2, 5, 6, 7, 12),
 (1, 2, 5, 6, 12),
 (1, 2, 5, 6, 12, 13),
 (1, 2, 5, 6, 13),
 (1, 2, 5, 6, 13, 

In [30]:
len(ineqT4)

145

In [26]:
from itertools import permutations


n=5

Perms=list(permutations(range(n)))

d5=merge(res5.get())


T5=[]
for key in d5.keys():
    T5.append(key[1])


ineqT5=[]
for t in T5:
    mode=True
    for p in Perms:
        pt=sorted(PermTuple(t,p))
        if isLectLess(t,pt):
            pass
        else:
            mode=False
            break
            
            
    if mode: 
        ineqT5.append(t)
              

In [27]:
len(ineqT5)

6310

In [29]:
print(ineqT5)

[(0, 1), (0, 1, 3), (0, 1, 3, 7), (0, 1, 3, 7, 15), (0, 1, 3, 15), (0, 1, 3, 15, 23), (0, 1, 7), (0, 1, 7, 11), (0, 1, 7, 11, 15), (0, 1, 7, 11, 15, 19), (0, 1, 7, 11, 19), (0, 1, 7, 11, 23), (0, 1, 7, 11, 23, 27), (0, 1, 7, 15), (0, 1, 7, 15, 27), (0, 1, 7, 27), (0, 1, 15), (0, 1, 15, 23), (0, 1, 15, 23, 27), (0, 3), (0, 3, 5), (0, 3, 5, 7), (0, 3, 5, 7, 9), (0, 3, 5, 7, 9, 15), (0, 3, 5, 7, 9, 15, 17), (0, 3, 5, 7, 9, 17), (0, 3, 5, 7, 9, 17, 25), (0, 3, 5, 7, 9, 23), (0, 3, 5, 7, 9, 23, 25), (0, 3, 5, 7, 9, 25), (0, 3, 5, 7, 15), (0, 3, 5, 7, 15, 25), (0, 3, 5, 7, 25), (0, 3, 5, 9), (0, 3, 5, 9, 15), (0, 3, 5, 9, 15, 17), (0, 3, 5, 9, 15, 17, 23), (0, 3, 5, 9, 15, 23), (0, 3, 5, 9, 17), (0, 3, 5, 9, 19), (0, 3, 5, 9, 19, 21), (0, 3, 5, 9, 19, 21, 23), (0, 3, 5, 9, 19, 21, 23, 25), (0, 3, 5, 9, 19, 21, 25), (0, 3, 5, 9, 19, 21, 27), (0, 3, 5, 9, 19, 21, 27, 29), (0, 3, 5, 9, 19, 23), (0, 3, 5, 9, 19, 23, 29), (0, 3, 5, 9, 19, 29), (0, 3, 5, 9, 23), (0, 3, 5, 9, 23, 27), (0, 3, 5, 9, 

In [31]:
len(ineqT2), len(ineqT3), len(ineqT4), len(ineqT5) #a(2), a(3), a(4), a(5)

(2, 13, 145, 6310)

## Direct filtering approach (including n=6)

In [2]:
%%cython --annotate
from math import log 
from itertools import permutations

cpdef iProcessRT(t,int i,int l,int nn1):
    """This function recursively extends an input tuple of integers, which represents 
    a set of sets (more precisily, a formal context), by adding to it an integer i (a new set).
    The length of a terminal tuple is constrained by l, while nn1 should be passed as 2**n-1. 
    The function checks whether the new tuple forms a unique closure system and returns the dictionary
    of the first elements of processed tuples as keys and values 1 for clousre system with
    interval closure propery and 0 otherwise."""
    
    count={}
    t=t[:]
    cdef int P
    cdef int lent=len(t)

    n=int(log(nn1+1,2))
    Perms=list(permutations(range(n)))
    
  

    
    #checking whether the context is reducible
    for j in range(lent-1,-1,-1):
        P=i
        #print("t[j],i",t[j],i,t[j]&i,)
        tji=t[j]&i
        if tji in t[0:j]:
            return count
            
        if tji==t[j]:
            #print("t[j],i",t[j],i,t[j]&i,)
            for h in range(j+1,lent):
                if t[j]&t[h]==t[j]:
                    P=P&t[h]
                    if P==t[j]:                        
                        return count
    t.append(i)
    
    # filtering by permuting and comparing w.r.t. lectically minimal system

    for p in Perms:
        pt=sorted(PermTuple(t,p))
        if not isLectLess(t,pt):
            return count
 
    if Check(t,nn1):
        count[(lent+1,t[0])]=1
    #else:
    #    count[(lent+1,t[0])]=0

    #recursive call
    cdef unsigned long long r
    #print(l)
    if l!=0:
        r=0
        for k in range(i+1,nn1):
            #print("Process(t,k,l-1,tuples,nn1)")
            #print("t",t,"k",k)
            ucount=iProcessRT(t,k,l-1,nn1)
            for key in ucount:
                if key in count:
                    count[key]+=ucount[key]
                else:
                    count[key]=ucount[key]
            #if r<0: print("r",r)
        return count
    else: 
        #print(tuples)
        #tuples.append(tuple(t))
       
        
        print("Branch process",t[0])
        return count

    
def isClosed(t,g):
    """Checks whether g is closed in t."""
    cont=[]
    for h in t:
        if g&h==g:
            cont.append(h)
    if len(cont)>0:
        res=cont[0]
        for h in cont:
            res=h&res
        return res==g
    else:
        return g==0
    
def Closure(t,g):
    """Returns closure of g in t."""
    res=~0
    for h in t:
        if g&h==g:
            res=h&res
    return res
    
    
def Check(t,nn1):
    """Checks whether {x,y}'' is subset of S is fulfilled for t."""
    t=t+[nn1]
    #print(t)
    n=int(log(nn1+1,2))
    members=[]
    members.extend([2**i for i in range(n)])
    powerset=range(nn1)
    lm=len(members)
    pairs=[members[i]|members[j] for i in range(lm) for j in range(i+1,lm)]
    closures={}
    for p in pairs:
        closures[p]=Closure(t,p)
    for s in powerset:
        mode=True
#         for i in range(lm):
#             for j in range(i+1,lm):
#                 p=members[i]|members[j]
        for p in pairs:   
            #cl=Closure(t,p)
            if p&s==p:
                if closures[p]&s!=closures[p]:
                        #print("False")
                        #print(p,cl,s)
                    mode=False
                    break
            #if mode==False: break           
                        
        if mode:
            if s not in members and Closure(t,s)!=s:
                #print("False")
                #print(t,s)
                #print(Closure(t,s))
                return False
                        
            
    return True


def PermBit(bit,perm):
    out=0
    for i in range(len(perm)):
        if (bit>>i) & 1==1:
            out=out | (1<<perm[i])
    return out

def PermTuple(t,perm):
    ret=[]
    for bit in t:
        ret.append(PermBit(bit,perm))
    return tuple(ret)


def isLectLess(a,b):
    #is A lectically smaller than B?
    for i in range(len(a)):
        if a[i]==b[i]:
            pass
        elif a[i]<b[i]:
            return True
        else:
            return False
    return True
        
    
def merge(r):
    """This is a value collector for the resulting list of dictionaries
    """
    merged={}
    for dic in r:
        for key in dic:
            if key in merged:
                merged[key]+=dic[key]
            else:
                merged[key]=dic[key]
    return merged

    

In [5]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count


n=5

#computing A334254 for n=6 by levels
if __name__ == "__main__":
    pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
   
     
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res5 = pool.starmap_async(iProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res5.get())# print the list of resulting dictionaries
    print(sum(merge(res5.get()).values()))#print the number of closure systems w.r.t. T1
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 30, 31), ([0], 2, 30, 31), ([0], 3, 30, 31), ([0], 4, 30, 31), ([0], 5, 30, 31), ([0], 6, 30, 31), ([0], 7, 30, 31), ([0], 8, 30, 31), ([0], 9, 30, 31), ([0], 10, 30, 31), ([0], 11, 30, 31), ([0], 12, 30, 31), ([0], 13, 30, 31), ([0], 14, 30, 31), ([0], 15, 30, 31), ([0], 16, 30, 31), ([0], 17, 30, 31), ([0], 18, 30, 31), ([0], 19, 30, 31), ([0], 20, 30, 31), ([0], 21, 30, 31), ([0], 22, 30, 31), ([0], 23, 30, 31), ([0], 24, 30, 31), ([0], 25, 30, 31), ([0], 26, 30, 31), ([0], 27, 30, 31), ([0], 28, 30, 31), ([0], 29, 30, 31), ([0], 30, 30, 31), ([1], 2, 30, 31), ([1], 3, 30, 31), ([1], 4, 30, 31), ([1], 5, 30, 31), ([1], 6, 30, 31), ([1], 7, 30, 31), ([1], 8, 30, 31), ([1], 9, 30, 31), ([1], 10, 30, 31), ([1], 11, 30, 31), ([1], 12, 30, 31), ([1], 13, 30, 31), ([1], 14, 30, 31), ([1], 15, 30, 31), ([1], 16, 30, 31), ([1], 17, 30, 31), ([1], 18, 30, 31), ([1], 19, 30, 31), ([1], 20, 30, 31), ([1], 21, 30, 31), ([1], 22, 30, 31), ([1], 23, 30, 31), ([1], 24, 30, 31), ([1], 25,

[{(2, 0): 1, (3, 0): 3, (4, 0): 6, (5, 0): 7, (6, 0): 2}, {}, {(2, 0): 1, (3, 0): 5, (4, 0): 16, (5, 0): 33, (6, 0): 36, (7, 0): 17, (8, 0): 2}, {}, {}, {}, {(2, 0): 1, (3, 0): 4, (5, 0): 11, (6, 0): 6, (4, 0): 8, (7, 0): 1}, {}, {}, {}, {}, {}, {}, {}, {(2, 0): 1, (3, 0): 1, (4, 0): 1, (5, 0): 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {(2, 1): 1, (3, 1): 9, (4, 1): 53, (5, 1): 207, (6, 1): 526, (7, 1): 782, (8, 1): 589, (9, 1): 205, (10, 1): 36, (11, 1): 2}, {(3, 1): 5, (4, 1): 32, (5, 1): 115, (6, 1): 251, (7, 1): 314, (8, 1): 194, (9, 1): 43, (10, 1): 4}, {}, {}, {(2, 1): 1, (3, 1): 10, (4, 1): 51, (5, 1): 166, (6, 1): 323, (7, 1): 314, (8, 1): 105, (9, 1): 12}, {(5, 1): 34, (6, 1): 34, (7, 1): 13, (8, 1): 1, (4, 1): 16, (3, 1): 3}, {}, {}, {}, {}, {}, {}, {(2, 1): 1, (3, 1): 4, (4, 1): 6, (5, 1): 8, (6, 1): 6}, {(5, 1): 1, (4, 1): 1, (3, 1): 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {(2, 1): 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {},

In [3]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count


n=6

#computing A334254 for n=6 by levels
if __name__ == "__main__":
    #pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
    pool = Pool(len(lt))
    
    print("lt=",len(lt))
    
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res6 = pool.starmap_async(iProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res6.get())# print the list of resulting dictionaries
    print(sum(merge(res6.get()).values()))#print the number of closure systems w.r.t. T1
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

lt= 1953
[([0], 1, 62, 63), ([0], 2, 62, 63), ([0], 3, 62, 63), ([0], 4, 62, 63), ([0], 5, 62, 63), ([0], 6, 62, 63), ([0], 7, 62, 63), ([0], 8, 62, 63), ([0], 9, 62, 63), ([0], 10, 62, 63), ([0], 11, 62, 63), ([0], 12, 62, 63), ([0], 13, 62, 63), ([0], 14, 62, 63), ([0], 15, 62, 63), ([0], 16, 62, 63), ([0], 17, 62, 63), ([0], 18, 62, 63), ([0], 19, 62, 63), ([0], 20, 62, 63), ([0], 21, 62, 63), ([0], 22, 62, 63), ([0], 23, 62, 63), ([0], 24, 62, 63), ([0], 25, 62, 63), ([0], 26, 62, 63), ([0], 27, 62, 63), ([0], 28, 62, 63), ([0], 29, 62, 63), ([0], 30, 62, 63), ([0], 31, 62, 63), ([0], 32, 62, 63), ([0], 33, 62, 63), ([0], 34, 62, 63), ([0], 35, 62, 63), ([0], 36, 62, 63), ([0], 37, 62, 63), ([0], 38, 62, 63), ([0], 39, 62, 63), ([0], 40, 62, 63), ([0], 41, 62, 63), ([0], 42, 62, 63), ([0], 43, 62, 63), ([0], 44, 62, 63), ([0], 45, 62, 63), ([0], 46, 62, 63), ([0], 47, 62, 63), ([0], 48, 62, 63), ([0], 49, 62, 63), ([0], 50, 62, 63), ([0], 51, 62, 63), ([0], 52, 62, 63), ([0], 53, 6

[{(2, 0): 1, (3, 0): 4, (4, 0): 13, (5, 0): 31, (6, 0): 52, (7, 0): 44, (8, 0): 18, (9, 0): 2}, {}, {(2, 0): 1, (3, 0): 7, (4, 0): 38, (5, 0): 166, (6, 0): 533, (7, 0): 1140, (8, 0): 1423, (9, 0): 889, (10, 0): 260, (11, 0): 40, (12, 0): 2}, {}, {}, {}, {(2, 0): 1, (3, 0): 7, (5, 0): 120, (6, 0): 308, (7, 0): 517, (8, 0): 482, (9, 0): 208, (4, 0): 33, (10, 0): 40, (11, 0): 4}, {}, {}, {}, {}, {}, {}, {}, {(2, 0): 1, (3, 0): 4, (6, 0): 26, (7, 0): 18, (8, 0): 6, (5, 0): 20, (9, 0): 1, (4, 0): 10}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {(2, 0): 1, (3, 0): 1, (4, 0): 1, (5, 0): 1, (6, 0): 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {(2, 1): 1, (3, 1): 12, (4, 1): 113, (5, 1): 868, (6, 1): 5339, (7, 1): 23988, (8, 1): 73241, (9, 1): 141900, (10, 1): 165667, (11, 1): 114525, (12, 1): 47430, (13, 1): 11529, (14, 1): 1542, (15, 1): 98, (16, 1): 2}, {(3, 1): 7, (4, 1): 77, (5, 1): 579, (6,

CPU times: user 4.02 s, sys: 7.63 s, total: 11.7 s
Wall time: 1d 22h 26min 25s


In [5]:
sum(merge(res6.get()).values())

2302154

In [4]:
merge(res6.get())

{(2, 0): 5,
 (3, 0): 23,
 (4, 0): 95,
 (5, 0): 338,
 (6, 0): 920,
 (7, 0): 1719,
 (8, 0): 1929,
 (9, 0): 1100,
 (10, 0): 300,
 (11, 0): 44,
 (12, 0): 2,
 (2, 1): 5,
 (3, 1): 59,
 (4, 1): 515,
 (5, 1): 3608,
 (6, 1): 19657,
 (7, 1): 77619,
 (8, 1): 207851,
 (9, 1): 354055,
 (10, 1): 364485,
 (11, 1): 222846,
 (12, 1): 81730,
 (13, 1): 17732,
 (14, 1): 2125,
 (15, 1): 125,
 (16, 1): 2,
 (4, 3): 465,
 (5, 3): 3471,
 (6, 3): 18581,
 (7, 3): 68449,
 (8, 3): 166274,
 (9, 3): 253514,
 (10, 3): 233372,
 (11, 3): 126496,
 (12, 3): 39602,
 (13, 3): 7025,
 (14, 3): 790,
 (16, 3): 7,
 (15, 3): 71,
 (3, 3): 46,
 (2, 3): 3,
 (5, 7): 587,
 (6, 7): 2202,
 (8, 7): 7642,
 (9, 7): 5951,
 (7, 7): 5294,
 (10, 7): 2454,
 (12, 7): 96,
 (11, 7): 595,
 (13, 7): 10,
 (14, 7): 1,
 (4, 7): 106,
 (3, 7): 12,
 (2, 7): 1,
 (6, 15): 45,
 (7, 15): 45,
 (8, 15): 25,
 (9, 15): 6,
 (10, 15): 1,
 (5, 15): 19,
 (4, 15): 5,
 (3, 15): 1,
 (6, 31): 1}